In [54]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%matplotlib inline
le = LabelEncoder()

In [55]:
sample = pd.read_csv('SF_crime/sampleSubmission.csv')
test = pd.read_csv('SF_crime/test.csv', index_col='Id')
test = test.rename(columns={'X': 'Longitude', "Y": "Latitude"})
test.Dates = pd.to_datetime(test.Dates)
test_keep = test
crime_in_sf = pd.read_csv('SF_crime/train.csv')
crime_in_sf.Dates = pd.to_datetime(crime_in_sf.Dates)
crime_in_sf = crime_in_sf.rename(columns={'X': 'Longitude', "Y": "Latitude",})
crime_in_sf = crime_in_sf.drop(['Resolution', 'Descript'], axis=1)
crime_train, crime_test = train_test_split(crime_in_sf, test_size=.4)

In [56]:
for column in test.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(test[column])
        test[column] = le.transform(test[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])
        crime_train[column] = le.transform(crime_train[column])

for column in crime_in_sf.columns.values:
    if column != 'Longitude' and column != 'Latitude':
        le.fit(crime_in_sf[column])        
        crime_test[column] = le.transform(crime_test[column])

/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MatthewBarnette/final_project_2/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
crime_train.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,Longitude,Latitude
431245,199741,21,4,9,2257,-122.413893,37.782347
202452,301329,20,6,4,15930,-122.432330,37.781429
634285,105273,1,6,9,18827,-122.414744,37.778719
697961,76950,7,3,3,6387,-122.419658,37.764221
569371,135427,35,1,6,12565,-122.466019,37.775271


In [58]:
categories = crime_train.Category
crime_train = crime_train.drop('Category', axis=1)

categories2 = crime_test.Category
crime_test = crime_test.drop('Category', axis=1)

In [59]:
dtrain = xgb.DMatrix(crime_train.as_matrix(),
                     label=categories)
dtest = xgb.DMatrix(crime_test.as_matrix(),
                    label=categories2)

In [61]:
param = {'bst:max_depth':6, 'objective':'multi:softprob', 'num_class':39}
param['nthread'] = 4
param['eval_metric'] = ['mlogloss', 'merror']

In [62]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [ ]:
num_round = 200
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=5)

Will train until train error hasn't decreased in 5 rounds.
Multiple eval metrics have been passed: 'merror' will be used for early stopping.

[0]	eval-mlogloss:3.103518	eval-merror:0.742942	train-mlogloss:3.096600	train-merror:0.740669
[1]	eval-mlogloss:2.926047	eval-merror:0.738201	train-mlogloss:2.916772	train-merror:0.735916
[2]	eval-mlogloss:2.812259	eval-merror:0.736877	train-mlogloss:2.800406	train-merror:0.734396
[3]	eval-mlogloss:2.729690	eval-merror:0.735539	train-mlogloss:2.715824	train-merror:0.733204
[4]	eval-mlogloss:2.669227	eval-merror:0.734807	train-mlogloss:2.653466	train-merror:0.732023
[5]	eval-mlogloss:2.621853	eval-merror:0.733580	train-mlogloss:2.604320	train-merror:0.730873
[6]	eval-mlogloss:2.584888	eval-merror:0.732817	train-mlogloss:2.565605	train-merror:0.729768
[7]	eval-mlogloss:2.555097	eval-merror:0.732322	train-mlogloss:2.533905	train-merror:0.728960
[8]	eval-mlogloss:2.531972	eval-merror:0.731442	train-mlogloss:2.509042	train-merror:0.728172
[9]	eval-mlo

In [ ]:
predictions = bst.predict(xgb.DMatrix(test.as_matrix()), output_margin=False)

In [ ]:
predictions = pd.DataFrame(predictions)

In [ ]:
le.fit(crime_in_sf.Category)
predictions.columns = le.inverse_transform(predictions.columns)

In [ ]:
pred_test = predictions

In [ ]:
predictions['Id'] = predictions.index

In [ ]:
def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame
predictions = order(predictions,['Id'])

In [ ]:
predictions.to_csv('predictions_XGB2.csv', index=False)

In [ ]:
def dummy_to_column(df):
    columns = df.columns.values
    df_t = df.T
    characters = []
    for column in range(len(columns)):
        for row in range(len(df[columns[column]])):
            if df[columns[column]][row] == '1':
                df[columns[column]][row] = columns[column]
    columns = df_t.columns.values
    characters = []
    for column in range(len(columns)):
        for row in range(len(df_t[columns[column]])):
            if df_t[columns[column]][row] in df_t.index:
                characters.append(df_t[columns[column]][row])
    return characters

def prob_to_column(df):
    df_t = df.T
    columns = df_t.columns.values
    probability = []
    for column in range(len(columns)):
        current_prob = 0
        for row in range(len(df_t[columns[column]])):
            if df_t[columns[column]][row] > current_prob:
                current_prob = df_t[columns[column]][row]
        probability.append(current_prob)
    n = 0
    df_t = df_t.applymap(str)
    for column in range(len(columns)):
        for row in range(len(df_t[column])):
            if df_t[column][row] == str(probability[n]):
                df_t[column][row] = '1'
        n+=1
    return df_t.T

In [ ]:
pred_results = pd.DataFrame(dummy_to_column(prob_to_column(pred_test)))